In [1]:
import os
import sys
project_root = os.getcwd()
sys.path.append(project_root + '/modules')
# Pipeline의 구조를 이해하기 쉽게 하기 위해 작성한 ipynb 파일입니다
from modules.Pipeline import pipe_eval
pipe = pipe_eval(verbose = True) # verbose를 False로 하면 print가 되지 않습니다.
# collection_name = 'chrdb.db',
# persist_directory = 'DB',
# mode = 'NaverCloudEmb'
# topk = 5
# verbose = True : answer과 score을 보여줍니다. 보고싶지 않다면 False로 하면 됩니다.
# default로 지정돼 있습니다. 클래스 선언 때 건네주면 바뀝니다.
pipe.setup(model = 'Exaone')
#                   가능한 모델 (default GPT)
#                   1. GPT
#                   2. Exaone
#                   3. Qwen14B
#                   4. Qwen32B
#                   5. HyperClova

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
생성된 DB가 있어 로드합니다.


/data/ephemeral/home/level4-nlp-finalproject-hackathon-nlp-09-lv3/src/modules/DB/chromadb_storing.py:127: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.db = Chroma(


Collection 'chrdb.db' loaded from 'modules/DB/Eval_DB_NaverCloudEmb'.
Total documents in collection 'chrdb.db': 891


Loading checkpoint shards: 100%|██████████| 7/7 [00:10<00:00,  1.43s/it]


In [4]:
query = 'SK하이닉스의 2025~2027년 주주환원 정책에서 변경된 고정배당금과 추가 배당금의 활용 방안은 무엇인가요?'
# query = 'NAVER의 분기별 실적 추이 및 전망에 대하여 알려줘'
# query = 'SK하이닉스의 2024년 매출에 대해 알려줘.'
# query = '로스데렌탈의 업종지수에 대한 비중확대, 중립, 비중축소의 기준은 무엇인가요?'
# validation데이터셋의 첫번째 question
retrieval_result = pipe.Q(query, mode = 'ensemble')
# bm25, dpr, ensemble 가능 (default 'ensemble')


page_content='SK하이닉스는 2024년부터 2026년까지의 주주환원 정책을 공개하며, 연간 고정 배당금을 1,200원에서 1,500원으로 인상하고, 잉여현금흐름(FCF)의 5%를 추가 배당금으로 지급하던 것을 중단하고 이를 재무 건정성 강화에 투입한다고 밝혔다. 이는 메모리 산업의 불확실성에 대비하여 안정적인 현금 흐름을 확보하려는 노력으로 평가된다.' metadata={'company_name': 'SK하이닉스', 'dir_route': './modules/datas/SK하이닉스_하나증권(2024.11.28)', 'file_name': 'SK하이닉스_하나증권(2024.11.28).pdf', 'id': 15, 'investment': '하나증권(2024.11.28)', 'original_content': 'SK하이닉스는 장 종료 후에 향후 3개년간 주주환원 쟁책을 공시했다. 기존 3개년 주주환 원 정책에서 변화가 생겼다. 변경된 부분의 핵심은 연간 고정배당금을 기존 1,200원에서 1,500원으로 상향하고, 기존 정책 하에서 지급하던 추가 배당금이었던 연간 Free Cash Flow의 5% 재원을 재무 건전성 강화에 우선 활용한다는 점이다. 아울러 정책 종료 후 총 재원 안에서 재무 건전성 목표가 달성되면 추가 환원을 실행하거나, 유의미한 수준의 Free Cash Flow 창출이 예상되는 경우에 조기 환원도 가능하다고 언급했다. 다시 설명하 자면 2025~2027년까지 3년 누적 Free Cash Flow의 50% 수준을 총 재원으로 설정하는 것은 기존과 동일하다. 다만, 기존에 매년 지급하던 FCF 내 5%의 추가 배당금을 앞으로 는 재무 건전성 강화로 사용하는 대신에 정책 종료 후 FCF 50% 수준 안에서 여유 재원을 기반으로 한 추가 환원을 결정한 것이다. 메모리 업황 특성상 변동성이 크기 때문에 예측 가능한 주주환원 규모를 선정하기 어렵기 때문에 3년 누적의 Free Cash Flo

In [3]:
# print(retrieval_result[1].metadata['table'])

In [5]:
answer = pipe.A(query, retrieval_result)
# query 와 retrieval 결과를 받아 answer을 냅니다.



다음 참고 문서를 반드시 활용하여 질문에 대한 명확하고 신뢰할 수 있는 답변을 작성하세요.

답변은 500자 이내로 작성하세요.
질문과 직접적으로 관련된 정보를 제공하세요.
사실적으로 정확한 정보만 포함하세요.
질문에 적절한 세부 정보를 포함하되, 명확하고 간결하게 작성하세요.
출처를 반드시 명시하세요.

질문: SK하이닉스의 2025~2027년 주주환원 정책에서 변경된 고정배당금과 추가 배당금의 활용 방안은 무엇인가요?
참고 문서: SK하이닉스_하나증권(2024.11.28).pdf, 1 page : SK하이닉스는 장 종료 후에 향후 3개년간 주주환원 쟁책을 공시했다. 기존 3개년 주주환 원 정책에서 변화가 생겼다. 변경된 부분의 핵심은 연간 고정배당금을 기존 1,200원에서 1,500원으로 상향하고, 기존 정책 하에서 지급하던 추가 배당금이었던 연간 Free Cash Flow의 5% 재원을 재무 건전성 강화에 우선 활용한다는 점이다. 아울러 정책 종료 후 총 재원 안에서 재무 건전성 목표가 달성되면 추가 환원을 실행하거나, 유의미한 수준의 Free Cash Flow 창출이 예상되는 경우에 조기 환원도 가능하다고 언급했다. 다시 설명하 자면 2025~2027년까지 3년 누적 Free Cash Flow의 50% 수준을 총 재원으로 설정하는 것은 기존과 동일하다. 다만, 기존에 매년 지급하던 FCF 내 5%의 추가 배당금을 앞으로 는 재무 건전성 강화로 사용하는 대신에 정책 종료 후 FCF 50% 수준 안에서 여유 재원을 기반으로 한 추가 환원을 결정한 것이다. 메모리 업황 특성상 변동성이 크기 때문에 예측 가능한 주주환원 규모를 선정하기 어렵기 때문에 3년 누적의 Free Cash Flow 선정은 동종 업체들의 정책과도 일맥상통하는 부분이다. 또한, SK하이닉스의 재무 건전성이 경쟁사들보 다 열위에 있던 점을 고려하면 금번 주주환원 정책 발표는 합리적인 결정이라 판단한다.
SK하이닉스_SK증권(2024.11.28).pdf, 

In [5]:
len(answer)

296

In [11]:
answer = pipe.QA(query, mode = 'ensemble')
# QA를 따로 내는 것이 아니라 한번에 정답을 얻을 수도 있습니다. 이후 정량평가에서는 QA를 사용하면 됩니다.

page_content='한화솔루션의 4분기 영업이익은 184억원으로 시장 기대치인 745억원보다 낮을 것으로 예상된다. 이는 기초소재 사업의 부진 때문이며, 태양광 사업에서는 흑자 전환이 기대된다. 특히 미국 인플레이션 감축법(IRA) 시행으로 인한 세액공제 혜택으로 태양광 모듈 판매량 증가와 발전사업 매출액 상승이 예상된다.' metadata={'company_name': '한화솔루션', 'dir_route': 'datas/한화솔루션_삼성증권(2024.10.30)', 'file_name': '한화솔루션_삼성증권(2024.10.30).pdf', 'id': 21, 'investment': '삼성증권(2024.10.30)', 'original_content': '4Q24 preview, 컨센서스 하회: 4Q 영업이익은 -184억원(+626억원QoQ)으로 컨센서스(745 억원, Fnguide) 하회 지속 예상되는데, 이는 기초소재 부진(영업이익 -321억원)에 기인. 태 양광 매출 및 영업이익은 각각 2.01조원(+75%QoQ) 및 116억원(흑자 전환QoQ; OPM +0.6%) 예상. 태양광 영업이익은 모듈 제조사업 -1,741억원(+193억원QoQ; OPM -20.7%), 발전사업 272억원(-12%QoQ; OPM +2.3%) 및 IRA AMPC 1,585억원(+30%QoQ)으로 구성 예상. 태양광 모듈 판매량은 전분기 이연 물량을 포함하여 크게 회복 전망(+58%QoQ vs 3Q24: Flat QoQ). 발전사업 매출도 일부 프로젝트 지연분 반영으로 급증 예상(1.17조원; +103%QoQ).', 'page': 1, 'type': 'paragraph'}
page_content='한화솔루션의 2024년 3분기 실적은 매출액 3.9조 원으로 전 분기 대비 6% 감소하였고 영업이익은 2,804억 원으로 전 분기 대비 23% 감소하였다. 이는 원재료 가격 상승 및 중국 수요 부진 

In [7]:
len(answer)

330

In [10]:
generated_answer, G_retrieval_score, G_generation_score = pipe.QA_eval(mode = 'ensemble', sampling = True)

# validation 데이터셋의 무작위 샘플을 가져와 G-eval로 평가할 수 있습니다.

page_content='카카오뱅크의 가계대출 포트폴리오는 꾸준히 개선되고 있으며, 주택담보대출 잔액은 2022년 3월 말 468억 원에서 2024년 6월 말 12조 5,738억 원으로 크게 증가하였고 이는 원화 대출금 중 29.5%를 차지한다.' metadata={'company_name': '카카오뱅크', 'dir_route': './modules/datas/카카오뱅크_한국신용평가(2024.09.13)', 'file_name': '카카오뱅크_한국신용평가(2024.09.13).pdf', 'id': 68, 'investment': '한국신용평가(2024.09.13)', 'original_content': '가 계대출 내 포트폴리오 다각화는 꾸준히 개선되고 있다. 주택담보대출의 경우 2022년 2월 출시 후 대출잔액은 2022년 3월 말 468억원에서 2022년 말 1조 1,957억원, 2024년 6월 말 12조 5,738억 원까지 증가하였으며 원화대출금 중 비중은 0.2%(2022년 3월 말)에서 4.3%(2022년 말), 29.5%(2024년 6월 말)까지 증가하였다.', 'page': 4, 'type': 'paragraph'}
page_content='카카오뱅크의 비대면 주택담보대출 규모가 빠른 속도로 확대되어 2024년 6월 말 기준 약 12조 5,700억 원이며, 개인사업자 신용대출 및 보증 대출 또한 빠르게 성장해 2023년 대출채권 성장률은 37.8%, 2024년 상반기에는 10.6%의 성장률을 보였다.' metadata={'company_name': '카카오뱅크', 'dir_route': './modules/datas/카카오뱅크_한국신용평가(2024.09.13)', 'file_name': '카카오뱅크_한국신용평가(2024.09.13).pdf', 'id': 124, 'investment':

In [13]:
eval_df = pipe.QA_eval(mode = 'ensemble', sampling = False) # sampling이 False일 경우, 모든 validation 데이터셋에 대해 평가를 진행합니다.
# 구현이 귀찮아서 일단 ./datas/g_eval_result_{model_name}.csv 경로에 저장되도록 하였습니다..

100%|██████████| 267/267 [1:29:53<00:00, 20.20s/it]

267 에 대한 평가를 마쳤습니다.


In [14]:
import pandas as pd
# retrieval G eval score과 generation G eval score의 점수 부분만 파싱합니다. (저장된 csv에는 전부 문자로 돼 있습니다.)

eval_df['generation_score'] = eval_df['generation_score'].apply(lambda x: x[x.rfind('/')-3:x.rfind('/')])
eval_df['retrieval_score'] = eval_df['retrieval_score'].apply(lambda x: x[x.rfind('/')-3:x.rfind('/')])
eval_df['generation_score'] = pd.to_numeric(eval_df['generation_score'], errors = 'coerce')
eval_df['retrieval_score'] = pd.to_numeric(eval_df['retrieval_score'], errors = 'coerce')

In [15]:
eval_df['generation_score'].describe()

count    267.000000
mean      28.026217
std        3.888878
min       12.000000
25%       30.000000
50%       30.000000
75%       30.000000
max       30.000000
Name: generation_score, dtype: float64

In [ ]:
retrieval_result[0].metadata['type']

In [8]:
table = """|---------------------|--------|--------|-------|-------|-------|
| 매출액              | 1,829  | 1,749  | 1,906 | 1,706 | 1,936 |
| 영업이익            | 230    | 83     | 106   | 123   | 139   |
| (증감률)            | -58.5  | -63.9  | 27.8  | 15.9  | 13.0  |
| 지배주주순이익      | -      | -      | -     | -     | -     |
| EPS                 | 10.73  | 10.71  | 11.5  | 12.0  | 12.5  |
| PER (H/L)           | 14.5   | 38.9   | 24.3  | 11.5  | 10.5  |
| PBR (H/L)           | 1.40   | 0.78   | 0.73  | -     | -     |
| EV/EBITDA (H/L)     | -      | -      | -     | -     | -     |
| 영업이익률          | -      | -      | -     | -     | -     |
| ROE                 | -      | -      | -     | -     | -     |"""

In [11]:
import pandas as pd
df = pd.read_csv('/data/ephemeral/home/clone/src/modules/datas/SK케미칼_DB금융투자(2024.11.06)/SK케미칼_DB금융투자(2024.11.06).csv')

In [21]:
import pandas as pd
import re

def markdown_to_df(markdown_table: str) -> pd.DataFrame:
    """
    마크다운 테이블을 Pandas DataFrame으로 변환하는 함수
    
    :param markdown_table: 마크다운 형식의 테이블 문자열
    :return: 변환된 Pandas DataFrame
    """
    lines = markdown_table.strip().split("\n")
    
    # 헤더 추출
    headers = [h.strip() for h in lines[0].split("|") if h.strip()]
    
    # 데이터 행 추출 (두 번째 줄은 구분선이므로 제외)
    data_rows = lines[2:]
    
    # 데이터 처리
    data = []
    for row in data_rows:
        values = [v.strip() for v in row.split("|") if v.strip()]
        data.append(values)
    
    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=headers)
    return df

In [28]:
df = pd.read_csv('/data/ephemeral/home/clone/src/modules/datas/SK케미칼_DB금융투자(2024.11.06)/SK케미칼_DB금융투자(2024.11.06).csv')



In [ ]:
q = 'SK하이닉스의 2024년 핵심사업에 대해 알려줘.'
result = pipe.Q(query)

In [ ]:
for i in result:
    print(i.metadata['type'])

In [ ]:
print(answer1)

In [ ]:
import os
import sys
project_root = os.getcwd()
sys.path.append(project_root + '/modules')
from Pipeline import pipe_service

pipe = pipe_service()
pipe.Q('SK하이닉스의 핵심사업에 대해 알려줘.')



In [3]:
import os
import sys
project_root = os.getcwd()
sys.path.append(project_root + '/modules')

In [4]:
from modules.utils import get_only_paragraphs
BASE_DIR = './modules/datas'
df = get_only_paragraphs(BASE_DIR=BASE_DIR)

In [5]:
df

,id,type,image_route,dir_route,file_name,page,investment,company_name,table,summary,original_content,iamge_route
0,6,paragraph,NaN,./modules/datas/CJ제일배당_신한투자증권(...,CJ제일배당_신한투자증권(2024.11.20).pdf,1,신한투자증권(2024.11.20),CJ제일배당,NaN,업종 Top Pick 의견 유지중인 CJ제일제당은 해외 영토 확장 및 글로벌 식품...,"업종 Top Pick 의견 유지. 2018년 헬스케어 매각을 통해 핵심 사업(식품,...",NaN
1,7,paragraph,NaN,./modules/datas/CJ제일배당_신한투자증권(...,CJ제일배당_신한투자증권(2024.11.20).pdf,1,신한투자증권(2024.11.20),CJ제일배당,NaN,"CJ제일제당 바이오 사업부가 6조원에 매각될 가능성이 있으며, 당사에서는 이를 24...",언론보도에 따르면 CJ제일제당 바이오 사업부 6조원에 매각 가능성 대두. 당사가 전...,NaN
2,8,paragraph,NaN,./modules/datas/CJ제일배당_신한투자증권(...,CJ제일배당_신한투자증권(2024.11.20).pdf,1,신한투자증권(2024.11.20),CJ제일배당,NaN,CJ제일제당은 과거 사례를 참고했을 때 매각 대금으로 유럽 시장 내 경쟁력 강화를 ...,과거 사례 참고 시 매각 대금은 유럽 시장 내 경쟁력 강화를 위한 M&A 에 쓰일 ...,NaN
3,9,paragraph,NaN,./modules/datas/CJ제일배당_신한투자증권(...,CJ제일배당_신한투자증권(2024.11.20).pdf,1,신한투자증권(2024.11.20),CJ제일배당,NaN,"CJ제일제당의 예상 매각 대금 3조 원 중 절반이 재무 구조 개선에 쓰이면, 부채 ...","예상 매각대금의 절반(3조원)이 재무구조 개선에 쓰일 경우, 연결기준 부 채비율과 ...",NaN
4,10,paragraph,NaN,./modules/datas/CJ제일배당_신한투자증권(...,CJ제일배당_신한투자증권(2024.11.20).pdf,1,신한투자증권(2024.11.20),CJ제일배당,NaN,CJ제일제당의 경우 해외 식품 매출이 기업 가치 평가 수준을 결정한다.,Valuation & Risk: 해외 식품 매출이 밸류에이션 레벨 결정,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,paragraph,NaN,./modules/datas/엘앤에프_DS투자증권(2024.1...,엘앤에프_DS투자증권(2024.11.04).pdf,1,DS투자증권(2024.11.04),엘앤에프,NaN,3분기 실적에서 고객사의 재고 조정과 유럽 수요 회복 지연으로 인해 엘앤에프의 영업...,3Q24 Re: 고객사 재고조정과 유럽 수요회복 지연으로 영업적자 지속,NaN
887,3,paragraph,NaN,./modules/datas/엘앤에프_DS투자증권(2024.1...,엘앤에프_DS투자증권(2024.11.04).pdf,1,DS투자증권(2024.11.04),엘앤에프,NaN,"엘앤에프의 3Q24P 매출액은 3,516억원으로 전년대비 72% 감소했고, 영업적자...","3Q24P 매출액 3,516억원(-36.6% QoQ, -72% YoY), 영업적자 ...",NaN
888,4,paragraph,NaN,./modules/datas/엘앤에프_DS투자증권(2024.1...,엘앤에프_DS투자증권(2024.11.04).pdf,1,DS투자증권(2024.11.04),엘앤에프,NaN,"4분기까지는 재고 조정 영향권이기 때문에, 엘앤에프의 흑자 전환은 내년에야 가능할 ...",4Q24까지 재고조정 영향권으로 예상됨에 따라 흑전은 내년이 될 전망,NaN
889,5,paragraph,NaN,./modules/datas/엘앤에프_DS투자증권(2024.1...,엘앤에프_DS투자증권(2024.11.04).pdf,1,DS투자증권(2024.11.04),엘앤에프,NaN,"엘앤에프의 4Q24F 매출액은 3,396억원이며 영업적자는 805억원으로 전망하며,...","4Q24F 매출액은 3,396억원(-3.4% QoQ, -48.4% YoY), 영업적...",NaN


In [6]:
from PIL import Image

# (30,1) 크기의 완전 투명한 이미지 생성 (RGBA 모드, 투명 = (0, 0, 0, 0))
transparent_img = Image.new("RGBA", (30, 1), (0, 0, 0, 0))

# 파일 저장
transparent_img.save("not.png")
